In [1]:
from enum import Enum
import sys, os
import nasdaqdatalink
import pandas as pd 
import numpy as np
from scipy.stats.mstats import gmean
from sqlalchemy import create_engine
from pathlib import Path

proj_root = Path(r"C:\dev\pynance\db\FundamentalRanksETL.py").resolve().parent.parent
sys.path.append(os.path.join(proj_root, 'lib'))
sys.path.append(os.path.join(proj_root, 'lib', 'equity'))

import nasdaq_data_link as nasdaq
from nasdaq_data_link import Sharadar
from numeric import custom_formatting

from calendar_dates import Calendar
from fundamentals import Fundamentals, Columns
cal = Calendar()

import warnings
warnings.filterwarnings('ignore')

C:\dev\pynance\lib



Two tables: One flat; One tall<br><br>
    CompFunBase - Company Fundamentals with a record indexed by Ticker and Calendar Date<br><br>
    CompFunRanks - Tall table indexed by Ticker, Industry, and Calendar Date with the companies ranks against their peer group (industry) <br>   Use this melted form to pivot and present ranks as a time series<br>
<br>
CompFunIndStats - descriptive statistics at the industry level for each date and metric
<br><br>
https://data.nasdaq.com/api/v3/datatables/SHARADAR/SF1?qopts.export=true&api_key=API_KEY



# Join equity fundamentals and static company info

In [2]:
# Fundamentals
fun = Fundamentals()
df_fun = fun.full_export(curl = False) # Set curl = True if data should be refreshed
df_fun = df_fun[df_fun.dimension == 'MRQ']

# Static Profile info
nasdaq.Nasdaq()
tick = nasdaq.Tickers()
df_prof = tick.full_export(curl = False) # Set curl = True if data should be refreshed

#Industry list
all_industies = df_prof.industry.unique().tolist()
# print(all_industies)

# Join
df = df_fun.set_index('ticker').merge(df_prof.set_index('ticker'), how='inner', left_index=True, right_index=True).reset_index()
df

,ticker,dimension,calendardate,datekey,reportperiod,lastupdated_x,accoci,assets,assetsavg,assetsc,...,name,exchange,sector,industry,scalemarketcap,scalerevenue,famasector,famaindustry,lastupdated_y,currency
0,A,MRQ,2017-03-31,2017-04-30,2017-04-30,2022-09-01,-477000000.0,8.016000e+09,NaN,3.800000e+09,...,AGILENT TECHNOLOGIES INC,NYSE,Healthcare,Diagnostics & Research,5 - Large,5 - Large,NaN,Measuring and Control Equipment,2022-09-01,USD
1,A,MRQ,2017-06-30,2017-07-31,2017-07-31,2022-09-01,-417000000.0,8.261000e+09,NaN,3.996000e+09,...,AGILENT TECHNOLOGIES INC,NYSE,Healthcare,Diagnostics & Research,5 - Large,5 - Large,NaN,Measuring and Control Equipment,2022-09-01,USD
2,A,MRQ,2017-09-30,2017-10-31,2017-10-31,2022-09-01,-346000000.0,8.426000e+09,NaN,4.169000e+09,...,AGILENT TECHNOLOGIES INC,NYSE,Healthcare,Diagnostics & Research,5 - Large,5 - Large,NaN,Measuring and Control Equipment,2022-09-01,USD
3,A,MRQ,2017-12-31,2018-01-31,2018-01-31,2022-09-01,-269000000.0,8.698000e+09,NaN,4.397000e+09,...,AGILENT TECHNOLOGIES INC,NYSE,Healthcare,Diagnostics & Research,5 - Large,5 - Large,NaN,Measuring and Control Equipment,2022-09-01,USD
4,A,MRQ,2018-03-31,2018-04-30,2018-04-30,2022-09-01,-310000000.0,8.784000e+09,NaN,4.525000e+09,...,AGILENT TECHNOLOGIES INC,NYSE,Healthcare,Diagnostics & Research,5 - Large,5 - Large,NaN,Measuring and Control Equipment,2022-09-01,USD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117182,ZYXI,MRQ,2022-06-30,2022-06-30,2022-06-30,2022-10-28,0.0,1.195750e+08,NaN,7.063000e+07,...,ZYNEX INC,NASDAQ,Healthcare,Medical Devices,3 - Small,2 - Micro,NaN,Medical Equipment,2022-07-29,USD
117183,ZYXI,MRQ,2022-09-30,2022-09-30,2022-09-30,2022-10-28,0.0,1.154350e+08,NaN,6.738200e+07,...,ZYNEX INC,NASDAQ,Healthcare,Medical Devices,3 - Small,2 - Micro,NaN,Medical Equipment,2022-07-29,USD
117184,NaN,MRQ,2020-12-31,2020-12-31,2020-12-31,2022-08-12,0.0,6.149349e+07,NaN,5.900875e+07,...,NANO LABS LTD,NASDAQ,Technology,Semiconductors,3 - Small,1 - Nano,NaN,Electronic Equipment,2022-08-12,USD
117185,NaN,MRQ,2021-12-31,2021-12-31,2021-12-31,2022-08-12,-2467327.0,9.098327e+08,NaN,8.934280e+08,...,NANO LABS LTD,NASDAQ,Technology,Semiconductors,3 - Small,1 - Nano,NaN,Electronic Equipment,2022-08-12,USD


# Subset for rank columns of interest

In [11]:
df = df[Columns.RANKS.value]

In [12]:
cols = ['ticker', 'calendardate'] + [c for c in df.columns if c not in ['ticker', 'calendardate']]
df = df[cols]

In [13]:
df.shape

(117187, 62)

In [ ]:
# df.columns.tolist()

# 1. load fundamentals to flat table

In [ ]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///C:\data\industry_fundamentals.db', echo=False)
cnxn = engine.connect()
df.to_sql(con=cnxn, if_exists='replace', name = 'CompFunBase', index = False) #Company Fundamentals base

In [ ]:
base = pd.read_sql("select * from CompFunBase where industry = 'Utilities - Regulated Electric' and ticker = 'DUK'", cnxn)
base

# 2. load ranks to tall table

CompFunRanks

In [ ]:
dates = df['calendardate'].unique().tolist()
industries =  df['industry'].unique().tolist()

In [ ]:
frames =[]
for date in dates[-6:]:
    for industry in industries:
        data = df[(df.calendardate == date) & (df.industry == industry)].set_index(['ticker','calendardate','industry'])
        ranks = data.rank(axis=1, pct=True, numeric_only = True).reset_index()
        melt = ranks.melt(id_vars = ['ticker', 'calendardate','industry'])
        frames.append(melt)
res = pd.concat(frames, axis=0)

In [ ]:
res

In [ ]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///C:\data\industry_fundamentals.db', echo=False)
cnxn = engine.connect()
res.to_sql(con=cnxn, if_exists='replace', name = 'CompFunRanks', index = False) #Company Fundamentals Ranks

In [ ]:
# res[(res.ticker == 'AMZN') & (res.variable == 'revenue')]

In [ ]:
rank = pd.read_sql("select * from CompFunRanks where ticker == 'AMZN'", cnxn)
rank

# 2b. Testing

In [4]:
x = df[(df.industry == 'Internet Retail') & (df.calendardate == '2022-06-30')]
# x = x[['revenue','gp','receivables','ebt','taxexp','capex']]
# x = pd.concat([x, pd.DataFrame([[np.nan,np.nan,np.nan,np.nan,np.nan,np.nan]], columns = ['revenue','gp','receivables','ebt','taxexp','capex'])])
x

,ticker,dimension,calendardate,datekey,reportperiod,lastupdated_x,accoci,assets,assetsavg,assetsc,...,name,exchange,sector,industry,scalemarketcap,scalerevenue,famasector,famaindustry,lastupdated_y,currency
4024,AKA,MRQ,2022-06-30,2022-06-30,2022-06-30,2022-08-10,-3.570600e+07,6.990710e+08,NaN,2.088000e+08,...,AKA BRANDS HOLDING CORP,NYSE,Technology,Internet Retail,3 - Small,3 - Small,NaN,Retail,2022-08-10,USD
6436,AMZN,MRQ,2022-06-30,2022-06-30,2022-06-30,2022-10-28,-4.782000e+09,4.197280e+11,NaN,1.336670e+11,...,AMAZON COM INC,NASDAQ,Technology,Internet Retail,6 - Mega,6 - Mega,NaN,Retail,2022-07-29,USD
7601,APRN,MRQ,2022-06-30,2022-06-30,2022-06-30,2022-08-08,0.000000e+00,2.160710e+08,NaN,1.086790e+08,...,BLUE APRON HOLDINGS INC,NYSE,Technology,Internet Retail,3 - Small,3 - Small,NaN,Retail,2022-08-08,USD
16106,BOXD,MRQ,2022-06-30,2022-06-30,2022-06-30,2022-08-11,-2.000000e+04,1.859790e+08,NaN,8.167100e+07,...,BOXED INC,NYSE,Technology,Internet Retail,3 - Small,2 - Micro,NaN,Retail,2022-08-11,USD
21954,CHWY,MRQ,2022-06-30,2022-07-31,2022-07-31,2022-08-31,0.000000e+00,2.368513e+09,NaN,1.505428e+09,...,CHEWY INC,NYSE,Technology,Internet Retail,5 - Large,4 - Mid,NaN,Retail,2022-08-31,USD
25599,CPNG,MRQ,2022-06-30,2022-06-30,2022-06-30,2022-08-12,-1.169100e+07,8.425203e+09,NaN,5.161399e+09,...,COUPANG INC,NYSE,Technology,Internet Retail,5 - Large,5 - Large,NaN,Retail,2022-08-12,USD
28437,CVNA,MRQ,2022-06-30,2022-06-30,2022-06-30,2022-08-04,0.000000e+00,1.050200e+10,NaN,5.491000e+09,...,CARVANA CO,NYSE,Technology,Internet Retail,5 - Large,5 - Large,NaN,Retail,2022-08-04,USD
30656,DIBS,MRQ,2022-06-30,2022-06-30,2022-06-30,2022-08-12,-3.980000e+05,2.120920e+08,NaN,1.770870e+08,...,1STDIBSCOM INC,NASDAQ,Technology,Internet Retail,3 - Small,2 - Micro,NaN,Retail,2022-08-12,USD
32293,DTC,MRQ,2022-06-30,2022-06-30,2022-06-30,2022-08-12,5.300000e+04,8.591630e+08,NaN,1.920480e+08,...,SOLO BRANDS INC,NYSE,Technology,Internet Retail,3 - Small,3 - Small,NaN,Recreation,2022-08-12,USD
33086,EBAY,MRQ,2022-06-30,2022-06-30,2022-06-30,2022-08-04,3.230000e+08,1.965300e+10,NaN,5.067000e+09,...,EBAY INC,NASDAQ,Technology,Internet Retail,5 - Large,5 - Large,NaN,Business Services,2022-08-04,USD


In [9]:
y = x.set_index(['ticker','calendardate']).rank(axis=0, pct=False, numeric_only = True, ascending = False, na_option='bottom', method='min').reset_index()
y

,ticker,calendardate,accoci,assets,assetsavg,assetsc,assetsnc,assetturnover,bvps,capex,...,intcov,paoutratio,taxrate,retentionratio,expnetincgrow,exproegrow,eqreinvestrate,expebitgrow,expgrowthrate,famasector
0,AKA,2022-06-30,32.0,16.0,1.0,20.0,15.0,1.0,17.0,21.0,...,15.0,2.0,10.0,23.0,14.0,28.0,23.0,11.0,18.0,1.0
1,AMZN,2022-06-30,36.0,1.0,1.0,1.0,1.0,1.0,4.0,33.0,...,16.0,2.0,7.0,30.0,9.0,29.0,30.0,5.0,28.0,1.0
2,APRN,2022-06-30,7.0,26.0,1.0,27.0,22.0,1.0,23.0,13.0,...,18.0,2.0,25.0,5.0,32.0,13.0,5.0,33.0,5.0,1.0
3,BOXD,2022-06-30,17.0,29.0,1.0,30.0,23.0,1.0,35.0,12.0,...,28.0,2.0,18.0,11.0,2.0,24.0,11.0,32.0,21.0,1.0
4,CHWY,2022-06-30,7.0,10.0,1.0,8.0,11.0,1.0,31.0,26.0,...,22.0,2.0,18.0,33.0,33.0,24.0,33.0,30.0,32.0,1.0
5,CPNG,2022-06-30,30.0,6.0,1.0,4.0,5.0,1.0,25.0,32.0,...,17.0,2.0,30.0,3.0,27.0,19.0,3.0,21.0,3.0,1.0
6,CVNA,2022-06-30,7.0,5.0,1.0,3.0,4.0,1.0,10.0,31.0,...,13.0,2.0,29.0,20.0,25.0,20.0,20.0,15.0,11.0,1.0
7,DIBS,2022-06-30,20.0,27.0,1.0,25.0,27.0,1.0,16.0,9.0,...,24.0,2.0,18.0,1.0,24.0,24.0,1.0,25.0,2.0,1.0
8,DTC,2022-06-30,5.0,15.0,1.0,22.0,13.0,1.0,9.0,18.0,...,21.0,2.0,14.0,22.0,15.0,31.0,22.0,12.0,17.0,1.0
9,EBAY,2022-06-30,1.0,2.0,1.0,5.0,2.0,1.0,5.0,29.0,...,28.0,34.0,3.0,32.0,3.0,34.0,32.0,1.0,29.0,1.0


# 3. Industry descriptive stats

In [ ]:
dates = df['calendardate'].unique().tolist()
industries =  df['industry'].unique().tolist()

In [ ]:
frames =[]
for date in dates[-6:]:
    for industry in industries:
        data = df[(df.calendardate == date) & (df.industry == industry)].set_index(['ticker','calendardate','industry'])
        stats = data.describe().reset_index()
        stats.rename(columns={'index':'stat'}, inplace = True)
        stats['calendardate'] = date
        stats['industry'] = industry
        melt = stats.melt(id_vars = ['calendardate','industry','stat'])
        frames.append(melt)
res = pd.concat(frames, axis=0)

In [ ]:
res

In [ ]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///C:\data\industry_fundamentals.db', echo=False)
cnxn = engine.connect()
res.to_sql(con=cnxn, if_exists='replace', name = 'CompFunIndStats', index = False) #Company Fundamentals Ranks